#정형 데이터와 비정형 데이터


*   정형 데이터 : csv 파일과 같이 가지런히 정리되어 있는 데이터, 어떤 구조로 이루어진 데이터
*   비정형 데이터 : 데이터베이스나 엑셀로 표현하기 어려운 데이터, (텍스트 데이터, 음성 데이터 등  )


---


#앙상블 학습
앙상블 학습 : 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘    
대부분 결정 트리를 기반으로 만들어져 있다.
##랜덤 포레스트
랜덤 포레스트 : 앙상블 학습의 대표주자중 하나로 안정적인 성능 덕분에 널리 사용되는 알고리즘


*   결정 트리를 랜덤하게 만들어 결정 트리의 ***숲***을 만든다.
*   그 다음 각 결정 트리의 예측을 사용해 최종 예측을 만든다.

랜덤 포레스트은 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, 이 데이터를 만드는 방법이 독특하다.    
예를 들어 1000개의 샘플이 들어있는 가방에서 100개의 샘플을 뽑는다 하면 1개를 뽑고 뽑았던 샘플을 다시 목록에 넣는 방식이다.   .


*   중복된 샘플을 뽑을 수 있는 방식이다.   
*   기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만든다
*   또한 각 노드를 분할할 때 전체 특성주엥 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는다.
*   분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택한다.







---

먼저 와인 데이터셋을 판다스로 불러오고 훈련 세트와 데이터 세트로 나눈다.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

다음으로 cross_valudate() 함수를 사용해 교차 검증을 수행한다. RandomForestClassifier는 기본적으로 100개의 결정 트리를 사용하며로 n_jobs 매개변수를 -1로 지정하여 모든 CPU 코어를 사용하는 것이 좋다.   
또한 return_train_score의 매개변수를 True로 지정하면 검증 점수 뿐만 아니라 훈련 세트에 대한 점수도 같이 반환한다.

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state = 42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


결과를 보면 훈련 세트에 과대적합되어있다.      
랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공한다.     
또한 결정 트리의 큰 장점중 하나인 특성 중요도를 계산한다.

In [5]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


두번째 특성인 당도에 대해 1절에서의 중요도보다 낮아진 것을 볼 수 있다. 이런 이유는 랜점 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문이다. 그 겨로가로 하나의 특성에 과도하게 집중하기 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다.

랜덤 포레스트에는 자체적으로 모델을 평가하는 점수를 얻을 수도 있다. 부트 스트랩에 포함되지 않고 남은 샘플들을 OOB라고 하는데 이 남은 샘플들을 사용해 검증 세트처럼 평가할 수 있다.

In [9]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


#엑스트라 트리
엑스트라 트리는 랜덤 포레스트와 매우 비슷하게 동작하는 알고리즘으로 기본적으로 100개의 결정 트리를 훈련한다.     
랜덤 포레스트와의 차이는 부트스트랩 샘플을 사용하지 않는다는 점이다. 결정 트리를 만들 때 전체 훈련 세트를 사용하는 것이다. 대신 노드를 분할할 때, 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다.

In [12]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state = 42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


결과는 랜덤 포레스트와 비슷하다. 보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 한다. 하지만 핸덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점이다.

In [16]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


엑스트라 트리 또한 결정 트리보다 당도에 대한 의존석이 작다.

#그레이디언트 부스팅
그레이디언트 부스팅은 깊이가 얕은 결정 트리를 사용하여 이전 트리의오차를 보완하는 방식으로 앙상블 하는 방법이다. 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.     
이전에 배운 경사 하강법을 사용하여 트리를 앙상블에 추가한다.

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


거의 과대적합 되지 않는다. 그래디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강하다.

In [21]:
gb=GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores=cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


결정 트리의 개수를 500개로 5배나 늘렸지만 과대적합을 잘 억제하고 있다.

In [23]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


#히스토그램 기반 그레이디언트 부스팅
히스토그램 기반 그레이디언트 부스팅은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘이다.

*   먼저 입력 특성을 256개의 구간으로 나눈다. 따라서 노드를 분할할 때, 최적의 분할을 매우 빠르게 찾을 수 있다.
*   256개의 구간 중엣 하나를 떼어 놓고 누락된 값을 위해서 사용한다. 입력에 누락된 특성이 있어도 따로 전처리할 필요가 없다.



In [25]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


과대적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능을 제공한다.     
히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산하기 위해 permutation_importance 함수를 사용하낟. 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한다. 계산한다. 훈련 세트 뿐만 아니라 테스트 세트에도 적용할 수 잇고, 사이킷런에서 제공하는 추정기 모델에 모두 사용할 수 있다.

In [27]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)


[0.08876275 0.23438522 0.08027708]


permutation_importance 함수가 반환하는 객체는 반복하여 얻은 특성 중요도, 평균, 표준편차이다. 특성 중요도는 다음과 같다.

In [28]:
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


그레이디언트 부스팅과 비슷하게 더 당도에 집중하고 있음을 알 수 있다. 그렇다면 HistGradientBoosiongClassifier를 사용해 테스트 세트에서의 성능을 최종적으로 확인해본다.

In [29]:
hgb.score(test_input, test_target)

0.8723076923076923

테스트세트에서 약 87%의 정확도를 얻었다. 단일 결정 트리보다 좋은 결과를 얻었다.

---

사이킷런 말고도 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 있다.
대표적인 라이브러리는 XGBoost이다.

In [30]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


다른 라이브러리로는 마이크로소프트에서 만든 LightGBM이 있다.

In [34]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
